In [ ]:
##### LOADING #####


# Load the single nucleus RNA Seq data
filename_sn = os.path.join(FILEPATHBASE,'02 Analysis', 'annData_ManualAnnotate', 'calc', 'retina_sn_combined.h5ad')
retinas_sn = ad.read_h5ad(filename_sn)

# Load the spatial RNA seq data
filename = os.path.join(FILEPATHBASE,'02 Analysis', 'annData_ManualAnnotate', 'calc', 'retinas_all.h5ad')
retinas_all = ad.read_h5ad(filename)
retinas_all.obs['batch'] = retinas_all.obs['dataset']
retinas_all.obs['batch'][retinas_all.obs['batch'] == 'R2'] = 'R1'
retinas_all.raw = retinas_all  # keep full dimension safe 
SampleKey = {"R1": "A22_3781", "R2": "A22_3781", "R3": "A23-914", "R4": "A23-1277", "R5": "A23-1279", "R6": "A23-1422", "R7": "A23-1425-M1", "R8": "A23-1152_OD"}
retinas_all.uns["SampleKey"] = SampleKey




st_adata = retinas_all.copy()
sc_adata = retinas_sn.copy()






#### FILTERING ######

## MMD: May need quite a bit of tweaking


#sc.pp.filter_genes(st_adata, min_cells=100) ### MMD: Not sure about this param
sc.pp.highly_variable_genes(
    st_adata,
    flavor="seurat_v3",
    n_top_genes=5000,         ### MMD: Not sure about this param
    layer="counts_scvi",
    batch_key='batch',
    subset=True,
    span=1
)


#sc.pp.normalize_total(st_adata, target_sum=10e4)
#sc.pp.log1p(st_adata)
#st_adata.raw = st_adata

# filter genes to be the same on the spatial data
intersect = np.intersect1d(sc_adata.var_names, st_adata.var_names)
st_adata = st_adata[:, intersect].copy()
sc_adata = sc_adata[:, intersect].copy()
st_adata = st_adata[st_adata.X.getnnz(1)>0]
sc_adata = sc_adata[sc_adata.X.getnnz(1)>0]

st_adata = st_adata.copy()
sc_adata = sc_adata.copy()

G = len(intersect)



##### FIT MODELS ######



# Train the CondSVCI model with the single nuc seq data
sc.pl.embedding(
    sc_adata,
    basis='X_mde',
    color=["CellType"],
    frameon=False,
    ncols=1,
)

CondSCVI.setup_anndata(sc_adata, layer="counts", labels_key="CellType")
sc_model = CondSCVI(sc_adata, weight_obs=False)
sc_model.view_anndata_setup()
sc_model.train()
sc_model.history["elbo_train"].iloc[5:].plot()
plt.show()





st_adata.layers['counts_scvi']
st_adata = st_adata[st_adata.layers['counts_scvi'].getnnz(1)>0]

#Integration of sc into spatial data with DestVI

st_adata = st_adata[st_adata.layers['counts_scvi'].getnnz(1)>0]
DestVI.setup_anndata(st_adata, layer="counts_scvi")
st_model = DestVI.from_rna_model(st_adata, sc_model)
st_model.view_anndata_setup()

st_model.train(max_epochs=2500)

#st_model.train(max_epochs=2500, plan_kwargs={"lr": 1e-5})

st_model.history["elbo_train"].iloc[10:].plot()
plt.show()

st_adata.obsm["proportions"] = st_model.get_proportions()
